In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

In [2]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADSK', 'AMD', 'AMZN', 'ANET', 'APPH', 
    'AXON', 'BBY','BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD',
    'DDOG', 'DKNG', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'EVGO', 'FTCH',
    'FUV', 'FVRR', 'GH', 'GMED', 'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD',
    'LULU','MDB', 'MEDP', 'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR',
    'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW', 'PINS', 'PTON', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE', 'SHOP', 'SOLO', 'SPLK', 'SQ',
    'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO', 'TWTR',
    'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [3]:
DL = '/Users/damiansp/Downloads'

In [4]:
!ls $DL

Portfolio_Positions_Feb-17-2022.csv Positions.csv


### E*Trade

In [5]:
out = pd.DataFrame(index=SYMBOLS)
out.head()

""
AAPL
ABMD
ABNB
ADSK
AMD


In [6]:
et_df = (
    pd.read_csv(f'{DL}/Positions.csv', index_col=0, skiprows=1)[['Market Value']]
      .rename(columns={'Market Value': 'ET'}))
et_df.index = map(lambda x: x.split()[0], et_df.index)
et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
#et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
ABMD,307.11
ABNB,183.6
AMZN,3093.05
ANET,387.68999999999994
AXON,2121.28


In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
check_symbols(et_df)

All clear


In [9]:
out = pd.concat([out, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,307
ABNB,184
ADSK,0
AMD,0


### Fidelity

In [10]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Feb-17-2022.csv'

In [11]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,3532.39,$1.00,$0.00,$3532.39,NaN,NaN,NaN,NaN,7.55%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,3.00,$307.11,-$18.31,$921.33,-$54.93,-5.63%,+$2.55,+0.27%,1.97%,$918.78,$306.26,Cash
2,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,8.00,$183.60,-$3.04,$1468.80,-$24.32,-1.63%,+$100.32,+7.33%,3.14%,$1368.48,$171.06,Cash
3,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,13.00,$112.37,-$5.32,$1460.81,-$69.16,-4.53%,+$140.40,+10.63%,3.12%,$1320.41,$101.57,Cash
4,218520300,ROLLOVER IRA,ANET,ARISTA NETWORKS INC COM USD0.0001,4.00,$129.23,-$3.21,$516.92,-$12.84,-2.43%,+$35.54,+7.38%,1.10%,$481.38,$120.35,Cash


In [12]:
def convert_value(s):
    return round(float(s.replace('$', '')))

In [13]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,ABMD,921
2,ROLLOVER IRA,ABNB,1469
3,ROLLOVER IRA,AMD,1461
4,ROLLOVER IRA,ANET,517
5,ROLLOVER IRA,BYND,112


In [14]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
ABMD,921
ABNB,1469
AMD,1461
ANET,517
BYND,112


In [15]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [16]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,0,0,2195,0
ABMD,307,921,307,0
ABNB,184,1469,184,0
ADSK,0,0,0,0
AMD,0,1461,0,0


In [17]:
out.to_csv(f'{DL}/clean.csv')

In [18]:
!open $DL/clean.csv

In [22]:
!rm $DL/$filename $DL/Postions.csv $DL/clean.csv
!ls $DL